In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import h5py
from nd2_to_caiman import np_arr_from_nd2
import labrotation.file_handling as fh
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import matplotlib.colors as mcolors
from math import floor, sqrt, atan2
from datetime import datetime
import json
from labrotation import json_util
import scipy
from scipy import ndimage
import datadoc_util
from statsmodels.nonparametric.smoothers_lowess import lowess
import pandas as pd
import seaborn as sns
import multiprocess as mp  # multiprocessing does not work with IPython. Use fork instead.
import os
import random  # for possible stochastic algorithms

In [ ]:
grid_shape = (8,8)

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")
print(data_docu_folder)

In [ ]:
ddoc = datadoc_util.DataDocumentation(data_docu_folder)
ddoc.loadDataDoc()

In [ ]:
onsets_h5_fpath = fh.open_file("Open extracted onsets and grid h5 file!")
print(onsets_h5_fpath)

In [ ]:
df_onsets = pd.read_hdf(onsets_h5_fpath)

In [ ]:
if "i_sz" in df_onsets.keys():
    n_sz = max(df_onsets.i_sz)+1
else:
    df_onsets.i_sz = 0
    n_sz = 1

In [ ]:
grid_data1 = [df_onsets[df_onsets.i_sz == i].pivot_table(index="row", columns="col", values="onset1", aggfunc="median") for i in range(n_sz)]
if "onset2" in df_onsets.columns:
    grid_data2 = [df_onsets[df_onsets.i_sz == i].pivot_table(index="row", columns="col", values="onset2", aggfunc="median") for i in range(n_sz)]
grid_datasz = [df_onsets[df_onsets.i_sz == i].pivot_table(index="row", columns="col", values="onset_sz", aggfunc="median") for i in range(n_sz)]

In [ ]:
I_SEIZURE = 1


f, ax = plt.subplots(figsize=(12, 12))
sns.despine(f, left=True, bottom=True)
#sns.scatterplot(x="x", y="y", hue="onset_sz", size="onset_sz",
#                sizes=[5, 40, 80, 160, 240, 320], linewidth=0,
#                data=onsets_df, ax=ax, palette=hues)
sns.scatterplot(x="x", y="y", hue="quantile1", size="onset1",
                sizes=(10,80), linewidth=0,
                data=df_onsets[df_onsets.i_sz == I_SEIZURE], ax=ax)
ax.invert_yaxis()  # invert to match imshow() and in general, nd2 videos: (0, 0) is top left corner
ax.set_xlim((0, 512))
ax.set_ylim((0, 512))
plt.show()

In [ ]:
quantiles1_df = [df_onsets[df_onsets.i_sz == i].groupby(["quantile1"], as_index=False).mean() for i in range(n_sz)]
if "quantile2" in df_onsets.keys():
    quantiles2_df = [df_onsets[df_onsets.i_sz == i].groupby(["quantile2"], as_index=False).mean() for i in range(n_sz)]
quantilessz_df = [df_onsets[df_onsets.i_sz == i].groupby(["quantile_sz"], as_index=False).mean() for i in range(n_sz)]

In [ ]:
# Move from nd2/video-style coordinate system (top left = (0, 0)) to usual plotting coordinate style (bottom left = (0, 0))
for i_df, q1_df in enumerate(quantiles1_df):
    df = q1_df
    df["y_mirrored"] = df.apply(lambda row: -1*row["y"], axis=1)
    quantiles1_df[i_df] = df
    
if "quantile2" in df_onsets.keys():
    for i_df, q2_df in enumerate(quantiles2_df):
        df = q2_df
        df["y_mirrored"] = df.apply(lambda row: -1*row["y"], axis=1)
        quantiles2_df[i_df] = df
    
for i_df, sz_df in enumerate(quantilessz_df):
    df = sz_df
    df["y_mirrored"]= df.apply(lambda row: -1*row["y"], axis=1)
    quantilessz_df[i_df] = df

In [ ]:
def get_dx(df, colname="x", quantile_colname="quantile1"):
    max_quantile = df[quantile_colname].max()
    min_quantile = df[quantile_colname].min()
    x1 = df[df[quantile_colname] == max_quantile][colname].values[0]
    x0 = df[df[quantile_colname] == min_quantile][colname].values[0]
    return x1-x0

In [ ]:
def xyToThetaR(vec):
    # given (x, y), return (theta, r)
    return (atan2(vec[1], vec[0]), sqrt(pow(vec[0], 2) + pow(vec[1], 2)))

# descartes coordinates
vec1 = [(get_dx(quantiles1_df[i], "x", "quantile1"), get_dx(quantiles1_df[i], "y_mirrored", "quantile1")) for i in range(n_sz)]
if "quantiles2_df" in locals():
    vec2 = [(get_dx(quantiles2_df[i], "x", "quantile2"), get_dx(quantiles2_df[i], "y_mirrored", "quantile2") ) for i in range(n_sz)]
vecsz = [(get_dx(quantilessz_df[i], "x", "quantile_sz"), get_dx(quantilessz_df[i], "y_mirrored", "quantile_sz")) for i in range(n_sz)]
# polar coordinates as (theta, r)
arrows = {}  # sz, sd1, sd2 order
vec1pol = [xyToThetaR(vec1el) for vec1el in vec1]
arrows["sd1"] = [(vec1polel[0], 2) for vec1polel in vec1pol]  # categorical length, sz > sd1 > sd2

if "quantiles2_df" in locals():
    vec2pol = [xyToThetaR(vec2el) for vec2el in vec2]
    arrows["sd2"] = [(vec2polel[0], 1) for vec2polel in vec2pol]
vecszpol = [xyToThetaR(vecszel) for vecszel in vecsz]
arrows["sz"] = [( vecszpolel[0], 3) for vecszpolel in vecszpol]


In [ ]:
arrows

In [ ]:
event_types = [ "sd2", "sd1", "sz"]
colors = {"sd2":"lightblue", "sd1":"blue", "sz":"darkblue"}
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='polar')
ax.set_ylim(0, 3)
ax.set_title(f"{os.path.split(onsets_h5_fpath)[-1]}", fontsize=20)
for event_type in event_types:
    if event_type in arrows.keys():
        for i_sz in range(n_sz):
            ax.annotate('', xy=arrows[event_type][i_sz], xytext=(0, 0),
                        arrowprops=dict(facecolor=colors[event_type], edgecolor='none', width=4, headwidth=20, alpha=0.9))
ax.set_rgrids(np.linspace(0, 3, num=len(event_types)+1))
ax.set_yticklabels([""] + event_types)

In [ ]:
# 431 432 433
# 434 435 436
# 437 438 439
# 4310 4311 4312

fig = plt.figure(figsize=(18,18))
nrows = 2*n_sz
if "grid_data2" in locals():
    ncols = 3
else:
    ncols = 2

i_fig = 1
for i_sz in range(n_sz):
    ax1 = fig.add_subplot(nrows, ncols, i_fig)
    ax1.set_title(f"sz", fontsize=20)
    sns.heatmap(grid_datasz[i_sz], annot=False, linewidths=.5, ax=ax1)
    i_fig += 1
    
    ax2= fig.add_subplot(nrows, ncols, i_fig)
    ax2.set_title(f"sd1", fontsize=20)
    sns.heatmap(grid_data1[i_sz], annot=False, linewidths=.5, ax=ax2)
    i_fig += 1

    if "grid_data2" in locals():
        ax3 = fig.add_subplot(nrows, ncols, i_fig)
        ax3.set_title(f"sd2", fontsize=20)
        sns.heatmap(grid_data2[i_sz], annot=False, linewidths=.5, ax=ax3)
        i_fig += 1


    event_types = [ "sz", "sd1", "sd2"]
    colors = {"sd2":"lightblue", "sd1":"blue", "sz":"darkblue"}


    for i_event, event_type in enumerate(event_types):
        if event_type in arrows.keys():
            ax = fig.add_subplot(nrows, ncols, i_fig, projection='polar')
            ax.set_ylim(0, 3)
            ax.annotate('', xy=arrows[event_type][i_sz], xytext=(0, 0),arrowprops=dict(facecolor=colors[event_type], edgecolor='none', width=4, headwidth=8, alpha=0.9))
            ax.set_rgrids(np.linspace(0, 3, num=len(event_types)+1))
            ax.set_yticklabels(["", "sd2", "sd1", "sz"])
            #ax.invert_yaxis()
            #ax.invert_xaxis()
            i_fig += 1
plt.show()